### Project 02: Create a GUI Notebook Program

Monodeep Chahal 
03/31/24

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [9]:
# imports
import tkinter as tk
from tkinter import filedialog, scrolledtext, ttk
import datetime
import json
import csv

# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        
        self.geometry("600x400") # set the default window size
        self.title('Notebook') #set the default window title
        self.notebook = [] # initialize an attribute named 'notebook' as an empty list
        
        # I have created 3 notes for demonstration
        # each note has a title, text, linkm and tags
        self.notes = [
            MakeNote("Basic Python Data Types", # title of note
             "Python's core data types include Numbers (integers, floats, complex numbers) for mathematical operations, Strings for textual data, Lists and Tuples for ordered collections (mutable and immutable, respectively), Dictionaries for key-value paired data, and Booleans representing True or False values. Understanding these types is essential for data manipulation in Python.", # text for note
             datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), # date and time
             "https://docs.python.org/3/library/stdtypes.html",  # link for reference
             "#Python, #Data Types, #Basics"),  # tags for the note
            
            MakeNote("Writing Functions in Python", 
             "Functions are defined using the def keyword, followed by a name and parameters. They encapsulate code blocks for specific tasks and can return values. Including a docstring for documentation is a good practice. Functions enhance code modularity and reusability.",
             datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
             "https://docs.python.org/3/tutorial/controlflow.html#defining-functions", 
             "#Python, #Functions, #Modularity"), 
            
            MakeNote("Introduction to Tkinter Widgets", 
             "Tkinter, Python's standard GUI library, uses widgets for creating interactive applications. Key widgets include Labels (for text display), Buttons (for actions), Entry (for text input), and Frames (to organize the layout). Manipulating these widgets allows for the development of complex GUI applications.",
             datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
             "https://docs.python.org/3/library/tk.html", 
             "#Tkinter, #GUI, #Widgets")       
]
        # initialize frames for layout management
        self.left_frame = ttk.Frame(self)
        self.left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.right_frame = ttk.Frame(self)
        self.right_frame.pack(side=tk.RIGHT, fill=tk.Y)

        # initialize buttons for creating and managing notes
        ttk.Button(self.right_frame, text='New Note', command=self.new_note).pack(fill=tk.X, padx=10, pady=5)
        ttk.Button(self.right_frame, text='Open Notebook', command=self.open_notebook).pack(fill=tk.X, padx=10, pady=5)
        ttk.Button(self.right_frame, text='Save Notebook', command=self.save_notebook).pack(fill=tk.X, padx=10, pady=5)
        ttk.Button(self.right_frame, text='Refresh Notes', command=self.refresh_notes).pack(fill=tk.X, padx=10, pady=5)

        # displays the initial set of notes
        self.refresh_notes()

    # method to display a note's content in a new window
    def view_note(self, note):
        ViewNoteForm(note)
        
    # update the display on notes in the left frame
    def display_notes(self):
        for widget in self.left_frame.winfo_children():
            widget.destroy()
        for note in self.notes:
            btn = ttk.Button(self.left_frame, text=note.title, command=lambda n=note: self.view_note(n))
            btn.pack(pady=2, padx=10, anchor='w')

    def refresh_notes(self):
        self.display_notes()
        
    # open a new window to create a new note
    def new_note(self):
        note_window = NoteForm(self, self.notebook, self.notes)
        return None
        
    # load notes from a file
    def open_notebook(self):
        filepath = filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("CSV files", "*.csv"), ("JSON files", "*.json")])
        if filepath:
            self.notes.clear()  # Clear existing notes before loading new ones
            if filepath.endswith('.json'):
                with open(filepath, "r") as file:
                    notes_data = json.load(file)
                    self.notes = [MakeNote(note["title"], note["text"], note["date_created"]) for note in notes_data]
            elif filepath.endswith('.csv'):
                with open(filepath, newline='') as csvfile:
                    reader = csv.DictReader(csvfile)
                    self.notes = [MakeNote(row['title'], row['text'], row['date_created']) for row in reader]
            elif filepath.endswith('.txt'):
                with open(filepath, "r") as file:
                    lines = file.readlines()
                    self.notes = [MakeNote(lines[i].strip(), lines[i + 1].strip(), lines[i + 2].strip()) for i in range(0, len(lines), 3)]
            self.refresh_notes()
    # save notes to a file
    def save_notebook(self):
        filetypes = [("Text file", "*.txt"), ("CSV file", "*.csv"), ("JSON file", "*.json")]
        filepath = filedialog.asksaveasfilename(filetypes=filetypes)
        if filepath:
            if filepath.endswith('.json'):
                with open(filepath, "w") as file:
                    json.dump([note.to_dict() for note in self.notes], file)
            elif filepath.endswith('.csv'):
                with open(filepath, 'w', newline='') as csvfile:
                    fieldnames = ['title', 'text', 'date_created']
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    writer.writeheader()
                    for note in self.notes:
                        writer.writerow(note.to_dict())
            elif filepath.endswith('.txt'):
                with open(filepath, "w") as file:
                    for note in self.notes:
                        file.write(f"{note.title}\n{note.text}\n{note.date_created}\n\n")

# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example
# the submit method calls the MakeNote class that transforms the the entered data into a new note object.

class NoteForm(tk.Toplevel):
    
    def __init__(self, master, notebook, notes): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window
        self.geometry("400x400")
        self.title("New Note")

        tk.Label(self, text="Title:").pack()
        self.title_entry = tk.Entry(self)
        self.title_entry.pack()

        tk.Label(self, text="Text:").pack()
        self.text_entry = scrolledtext.ScrolledText(self, height=10)
        self.text_entry.pack()
        
        tk.Label(self, text="Link:").pack()
        self.link_entry = tk.Entry(self)
        self.link_entry.pack()

        tk.Label(self, text="Tags (comma-separated):").pack()
        self.tags_entry = tk.Entry(self)
        self.tags_entry.pack()
        
        # creating a button to save the new note
        tk.Button(self, text="Save Note", command=self.save_note).pack()

    # save the new note
    def save_note(self):
        title = self.title_entry.get()
        text = self.text_entry.get("1.0", tk.END).strip()
        link = self.link_entry.get()  # Capture link input
        tags = self.tags_entry.get()  # Capture tags input
        date_created = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_note = MakeNote(title, text, date_created, link, tags)
        self.master.notes.append(new_note)
        self.master.refresh_notes()
        self.destroy()
        
    def submit(self):
        
        new_note = MakeNote(note_dict)
        self.notes.append(new_note)
        print(self.notes)
        return None

    
# The MakeNote class takes a dictionary containing the data entered into the form window,
# and transforms it into a new note object.
# At present the note objects have attributes but no methods.

class MakeNote():
    def __init__(self, title, text, date_created, link='', tags=''):
        self.title = title
        self.text = text
        self.date_created = date_created
        self.link = link # allows user to add a link
        self.tags = tags # allows user to add tags
        
    def note_dict(self):
        return{"title": self.title, 
               "text":self.text, 
               "date_created": self.date_created,
               "link": self.link,
               "tags":self.tags
              }

# creating a simple window to view a note
class ViewNoteForm(tk.Toplevel):
    def __init__(self, note):
        super().__init__()
        self.title(note.title)
        self.geometry("400x300")
        
        # display the title
        tk.Label(self, text="Title:").pack()
        title_label = tk.Label(self, text=note.title)
        title_label.pack()

        # display the text
        tk.Label(self).pack()
        text_label = scrolledtext.ScrolledText(self, height=10, width=50)
        text_label.pack()
        text_label.insert(tk.END, note.text)
        text_label.config(state='disabled')
        
        # display the link
        tk.Label(self, text="Link:").pack()
        link_label = tk.Label(self, text=note.link)
        link_label.pack()

        # display the tags
        tk.Label(self, text="Tags:").pack()
        tags_label = tk.Label(self, text=note.tags)
        tags_label.pack()

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop()

#### Print your three notes below

In [10]:
# print your notes here
# Note 1: Basic Python Data Types
# Python's core data types include Numbers (integers, floats, complex numbers) for mathematical operations, Strings for textual data, Lists and Tuples for ordered collections (mutable and immutable, respectively), Dictionaries for key-value paired data, and Booleans representing True or False values. Understanding these types is essential for data manipulation in Python. 
                     
# Note 2: Writing Functions in Python
# Functions are defined using the def keyword, followed by a name and parameters. They encapsulate code blocks for specific tasks and can return values. Including a docstring for documentation is a good practice. Functions enhance code modularity and reusability.

# Note 3: Introduction to Tkinter Widgets", 
# Tkinter, Python's standard GUI library, uses widgets for creating interactive applications. Key widgets include Labels (for text display), Buttons (for actions), Entry (for text input), and Frames (to organize the layout). Manipulating these widgets allows for the development of complex GUI applications.
 